# Проект: Идентификация пользователей по посещенным веб-страницам

**Цель проекта:** Научиться идентифицировать пользователей на основе паттернов поведения в Интернете (через анализ последовательностей посещенных ими сайтов).

**Задачи проекта:**
1. Преобразовать данные в формат, подходящий для анализа и решения задачи машинного обучения
2. Выделить подвыборку, сформировать набор показателей для анализа и провести первичный анализ данных, оценить возможность выявления характерного поведения
3. Решить задачу машинного обучения с помощью нескольких алгоритмов
4. Выбрать алгоритм/ансамбль алгоритмов, дающий лучший результат

**Затрагиваемые области Machine Learning:** Supervised Learning, Sequential Pattern Mining

---

## Описание данных

Исходные данные взяты из [статьи](http://ceur-ws.org/Vol-1703/paper12.pdf) "A Tool for Classification of Sequential Data" и представляют собой набор файлов (по 1 файлу на каждого пользователя) с информацией о посещенных пользователями сайтах и временными метками перехода на каждую страницу. Пример:
<center>user0001.csv</center>
<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg .tg-yw4l{vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-031e">timestamp</th>
    <th class="tg-031e">site</th>
  </tr>
  <tr>
    <td class="tg-031e">00:00:01</td>
    <td class="tg-031e">vk.com</td>
  </tr>
  <tr>
    <td class="tg-yw4l">00:00:11</td>
    <td class="tg-yw4l">google.com</td>
  </tr>
  <tr>
    <td class="tg-031e">00:00:16</td>
    <td class="tg-031e">vk.com</td>
  </tr>
  <tr>
    <td class="tg-031e">00:00:20</td>
    <td class="tg-031e">yandex.ru</td>
  </tr>
</table>

Для дальнейшего анализа данные из файлов преобразуются в единый датасет, содержащий информацию о пользовательских сессиях, где под сессией понимается последовательность из N сайтов, подряд идущих в исходных файлах. При этом наименования всех сайтов, встречающихся в выборке кодируются и помещаются в ячейки датасета. Пример соответствующий таблице выше (N = 2, vk.com закодирован как 1, google.com - 2, yandex.ru - 3):
<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg .tg-s6z2{text-align:center}
.tg .tg-baqh{text-align:center;vertical-align:top}
.tg .tg-hgcj{font-weight:bold;text-align:center}
.tg .tg-amwm{font-weight:bold;text-align:center;vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-hgcj">session_id</th>
    <th class="tg-hgcj">site1</th>
    <th class="tg-hgcj">site2</th>
    <th class="tg-amwm">user_id</th>
  </tr>
  <tr>
    <td class="tg-s6z2">1</td>
    <td class="tg-s6z2">1</td>
    <td class="tg-s6z2">2</td>
    <td class="tg-baqh">1</td>
  </tr>
  <tr>
    <td class="tg-s6z2">2</td>
    <td class="tg-s6z2">1</td>
    <td class="tg-s6z2">3</td>
    <td class="tg-baqh">1</td>
  </tr>
</table>

Поскольку такой датасет содержит не вещественные, а категориальные признаки, строить модель непосредственно на нем нельзя. Требуется еще один этап преобразования - переход к идее мешка слов из анализа текстов и, соответственно, переход к разреженной матрице, где строкам соответствуют сессии из N сайтов, столбцам – индексы сайтов (их кодировки), а на пересечении строки и столбца стоит число k – cколько раз соответствующий сайт встретился в соответствующей сессии. Это будет сделано перед построением модели с использование библиотеки Scipy (функция csr_matrix).

Далее будет использоваться датасет по 10 пользователям.

---

## Первичный анализ данных

Первичный анализ проводился при N=10 и скользящим окном в 10 сайтов (значит, что каждая строка исходных данных попадет только в одну сессию, если скользящее окно будет меньше длины сессии, сессии будут перекрываться).

В ходе него были проведены анализ распределения количества уникальных сайтов в сессиях, распределение частот посещения сайтов, проверка нормальности распределения целевого признака, вариативность поведения пользователей; определены наиболее часто посещаемые сайты.

### Визуальный анализ признаков

Далее генерировались признаки, характеризующие поведение пользователей и анализировалось распределение признаков отдельно для каждого из пользователей. Это позволило выделить признаки, существенно варьирующиеся между пользователями и, соответственно, способными помочь в решении поставленной задачи.

***Важно, что из графиков следует наличие существенных отличий в поведении пользователей, соответственно, скорее всего, получится построить модель, идентифицирующую пользователей, с высоким уровнем качества.***

---

## Применение алгоритмов машинного обучения 

### Предобработка данных 

Датасет был предварительно преобразован в разреженную матрицу и разобит на обучающую и тестовую выборки в соотношении 0.7 и 0.3, также использовалась кросс-валидация для получения более объективной оценки качества (стратифицированная, 3-кратная, с перемешиванием).

### Метрики качества

Для сопоставимости результатов необходимо использовать минимальное количество метрик оценки качества для задач классификации. Одной из наиболее прозрачных и легко интерпретируемых метрик является **ROC AUC**. Это площадь под ROC-кривой, где по оси Х откладываются значения метрики 

$$False Positive Rate (FPR) = FP / (FP + TN),$$ а по оси Y - $$True Positive Rate (TPR) = TP / (TP + FN),$$ где 
- FP - это количество ложных срабатываний алгоритма (алгоритм отнес к классу 1, хотя не должен был), 
- TP - количество верных срабатываний алгоритма, 
- FN - количество ложных пропусков (алгоритм отнес к классу 0, когда должен был отнести к классу 1), 
- TN - количество верных пропусков (верно отнесено к классу 0). 

Эта метрика удобна тем, что имеет ограниченный диапазон значений, которые она способна принимать, и является легко интерпретируемой: варьируется от 0 до 1, чем выше значение, тем выше качества, уровень 0.5 соответствует предсказаниям на уровне случайных чисел.
Кроме того, она может принимать на вход как сами предсказанные метки классов, так и их вероятности. 
Но у нее есть и ограничение - она не может использоваться в задачах многоклассовой классификации (а у нас 10 пользователей, т.е. 10 классов), поэтому попробуем решить задачу в двух плоскостях: 

1. сравнивая 10 пользователей между собой. Здесь будет использоваться метрика **accuracy** *(доля правильно предсказанных меток классов, варьируется от 0 до 1, где 1 соответствует идеальной модели, не умеет работать с вероятностями и не чувствительна к несбалансированности выборки)*;
2. сравнивая одного пользователя со всеми остальными. Здесь мы сможем перейти к метрике **ROC AUC**.

### Выбор наиболее результативных алгоритмов

Были использованы 6 алгоритмов:
- KNeighborsClassifier
- RandomForestClassifier
- LogisticRegression
- Линейный SVM (LinearSVC)
- SGDClassifier
- Vowpal Wabbit

В итоге для дальнейшей работы выбраны те, которые показали наилучшее качество при предсказании вероятностей меток классов.

### Добавление к модели дополнительных параметров

К модели добавлены ранее отобранные признаки, чтобы понять, улучшит ли это качество предсказания меток классов.

### Настройка параметров моделей

Попытка улучшить результат путем оптимизации параметров моделей.

***В реультате модели были проранжированы по качеству полученных результатов.***
***Полученное решение имеет достаточно высокое качество (0.94790).***

---

### Применимость 

Такого рода задачи могут решаться для обеспечения безопасности (снижения вероятности или продолжительности несанкционированного использования аккаунтов пользователя), группировки аккаунтов или устройств (например, если пользователь может сидеть в интернете с рабочего компьютера, домашнего ноутбука и с телефона) и, как следствие, получения возможности проведения более глубокого анализа поведения потребителей услуг.

### Что еще можно было сделать 

1. entity recognition (преобразование наименований сайтов для исключение дубликатов, например, www.google.fr и www.google.com - 2 самых часто встречающихся сайта в датасете)
2. уделить больше внимания feature engineering и проверять влияние признаков на модель с помощью алгоритмов feature selection
3. примерение методов объединения моделей в ансамбли
4. выбор параметров датасета: количества сайтов в сессии и ширины скользящего окна
5. подбор параметров для случайного леса (с Tf-idf) - это долго
6. применение модели к большей выборке (150 или 3000 пользователей)